In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

categories_schema = StructType ([
  StructField("categoryID", IntegerType(),True),
  StructField("categoryName", StringType(), True),
  StructField("description", StringType(), True)
])

df = spark.read.format('csv').option("header",True).schema(categories_schema).load("/Volumes/learning/test_schema/managed_volume/categories")
# If you know the table does not already exist, you can call this instead:
df.write.saveAsTable("learning.test_schema.categories_delta_py");

# Create the table if it does not exist. Otherwise, replace the existing table.
#df.writeTo("learning.test_schema.categories_delta_py").createOrReplace();

##### read
df = spark.read.table("learning.test_schema.categories_delta_py")
df.show();
display(df);

###### update
from delta.tables import *
categories_dlt = DeltaTable.forName(spark,"learning.test_schema.categories_delta_py")
categories_dlt.update(
    condition = "categoryID = 2",
    set = { "categoryName": "'Updated'" }
)
##### delete
from delta.tables import *
categories_dlt = DeltaTable.forName(spark, "learning.test_schema.categories_delta_py")
categories_dlt.delete("categoryID = 2")

### history
categories_dlt = DeltaTable.forName(spark, "learning.test_schema.categories_delta_py")
display(categories_dlt.history())

## time travel
categories_dlt = DeltaTable.forName(spark, "learning.test_schema.categories_delta_py")
deltaHistory = categories_dlt.history()
display(deltaHistory.where("version == 1"))

## time travel read
table_name = "learning.test_schema.categories_delta_py"
df = spark.read.option('versionAsOf', 0).table(table_name)
display(df)
# optimize
categories_dlt = DeltaTable.forName(spark, "learning.test_schema.categories_delta_py")
categories_dlt.optimize().executeCompaction()
categories_dlt.vacuum()



categories_dlt.optimize().executeZOrderBy("categoryName")